In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Is this notebook running on Colab or Kaggle?
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
    if IS_KAGGLE:
        print("Go to Settings > Accelerator and select GPU.")

# Common imports
import numpy as np
import pandas as pd
import sklearn.preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import os
import datetime as dt
from pathlib import Path

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [ ]:
import cudf
import cupy as cp
import cuml

In [ ]:
# sub=cudf.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')

In [ ]:
# trans_train = cudf.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')
# trans_train.head()

In [ ]:
# sub_trans = trans_train[["customer_id" , "article_id"]]
# submission = sub[["customer_id"]]

In [ ]:
# sub_trans['customer_id'] = sub_trans['customer_id'].str[-16:].str.hex_to_int().astype('int64')
# submission['customer_id'] = submission['customer_id'].str[-16:].str.hex_to_int().astype('int64')
# sub_trans.head()

In [ ]:
# submission.tail()

In [ ]:
# cust_age= cudf.merge( submission['customer_id'], sub_trans[['customer_id' , 'article_id']] , on=['customer_id'] , how = "left")
# cust_age

In [ ]:
# sub=pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')

In [ ]:
################Code Start

In [ ]:
trans_train=pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')
trans_train['article_id'] = trans_train.article_id.astype('int32')

In [ ]:
submission = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')

In [ ]:
submission = submission["customer_id"]

In [ ]:
sub_sub = submission.drop_duplicates(keep = "first")

In [ ]:
sub_sub

In [ ]:
sub_trans = trans_train[["customer_id","article_id"]]
sub_trans

In [ ]:
sub_no_dup_trans = sub_trans.drop_duplicates('customer_id')
sub_no_dup_trans

In [ ]:
sub_train_cust = pd.merge(sub_no_dup_trans,sub_sub, on='customer_id', how='right' , indicator = True)
sub_train_cust

In [ ]:
print(len(sub_sub))
len(sub_train_cust)


In [ ]:
del sub_sub 
del trans_train
del submission
del sub_no_dup_trans

In [ ]:
import gc
gc.collect

In [ ]:
arti_train=pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv')

In [ ]:
sub_article_id = sub_train_cust.merge(arti_train, on='article_id', how='left').fillna(np.nan)
sub_article_id = sub_article_id[["customer_id","article_id",'product_type_no','graphical_appearance_no','colour_group_code']]
sub_article_id

In [ ]:
print(len(sub_train_cust))
len(sub_article_id)

In [ ]:
cus_train=pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/customers.csv')

In [ ]:
sub_cust_id = sub_article_id.merge(cus_train, on='customer_id', how='left')
sub_cust_id = sub_cust_id[["customer_id","age","article_id",'product_type_no','graphical_appearance_no','colour_group_code']]
sub_cust_id

In [ ]:
sub_cust_id.to_csv('submission_data.csv' , index = False)

In [ ]:
#########Code End